In [0]:
# Databricks notebook source
"""
# 00_generate_mock_data
Descrição:
    1. Configura o ambiente (Cria Catálogo/Schema/Volume se necessário).
    2. Gera dados sintéticos de clientes em formato JSON para simular um sistema transacional.
"""

import random
import json
import time
import os
from datetime import datetime, timedelta
import uuid

In [0]:
# --- CONFIGURAÇÃO MANUAL ---
# Pode ser alterado manualmente para simular um passado específico
sim_date_string = "2025-03-01" 

base_date = datetime.strptime(sim_date_string, "%Y-%m-%d")
print(f"🕒 GERANDO DADOS PARA: {base_date}")
# ... (resto do código igual) ...

In [0]:
# INATIVO
# # --- CONFIGURAÇÃO DE PARAMETROS ---
# # Permite passar uma data específica para simular o passado (Formato YYYY-MM-DD)
# dbutils.widgets.text("simulation_date", "")
# sim_date_param = dbutils.widgets.get("simulation_date")

# if sim_date_param:
#     # Se passamos data, usamos ela (ex: 2025-11-01)
#     base_date = datetime.strptime(sim_date_param, "%Y-%m-%d")
#     print(f"🕒 MODO SIMULAÇÃO: Gerando dados com data base: {base_date}")
# else:
#     # Se não, usa agora
#     base_date = datetime.now()
#     print(f"🕒 MODO REAL-TIME: Gerando dados com data atual: {base_date}")

In [0]:
#--- 1. SETUP DE AMBIENTE (Infraestrutura) ---

catalog = "main"
schema = "default"
volume = "lakehouse"
volume_path = f"/Volumes/{catalog}/{schema}/{volume}"
raw_path = f"{volume_path}/raw_customers"

os.makedirs(raw_path, exist_ok=True)

# --- GERADOR ---
def get_random_customer(ref_date):
    # IDs fixos entre 1001 e 1050 para garantir que REPITAM e gerem histórico no SCD2
    # IDs entre 1051 e 1100 aleatórios para simular NOVOS clientes
    
    if random.random() < 0.7: # 70% de chance de ser cliente antigo atualizando dados
        customer_id = random.randint(1001, 1050)
    else:
        customer_id = random.randint(1051, 1100 + int(time.time())%100) # Novos

    cities = ["São Paulo", "Rio de Janeiro", "Belo Horizonte", "Curitiba", "Recife", "Porto Alegre"]
    
    return {
        "customer_id": customer_id,
        "name": f"Cliente {customer_id}",
        "email": f"cliente{customer_id}@email.com",
        "city": random.choice(cities), # A cidade muda aleatoriamente, gerando o SCD2
        "signup_date": (ref_date - timedelta(days=random.randint(0, 30))).strftime("%Y-%m-%d"),
        "phone": f"(11) 9{random.randint(1000,9999)}-{random.randint(1000,9999)}",
        "update_ts": ref_date.isoformat() # A data chave para o nosso teste
    }

In [0]:
# --- 2. CONFIGURAÇÃO DO GERADOR ---

NUM_FILES = 2
RECORDS_PER_FILE = 100

for i in range(NUM_FILES):
    batch_data = [get_random_customer(base_date) for _ in range(RECORDS_PER_FILE)]
    
    # Nome do arquivo com timestamp simulado para não sobrescrever
    ts_file = int(base_date.timestamp())
    file_name = f"customers_{ts_file}_{i}.json"
    file_path = f"{raw_path}/{file_name}"
    
    with open(file_path, "w") as f:
        json.dump(batch_data, f)
        
    print(f"Arquivo gerado: {file_name}")

print("✅ Dados gerados com sucesso para a data simulada.")

In [0]:
# --- 3. FUNÇÕES GERADORAS (Mock Data) ---

def get_random_customer():
    """Gera um cliente falso com dados aleatórios."""
    customer_id = random.randint(1001, 1100) # Simula base de 100 clientes recorrentes
    
    # Lista de cidades para variar
    cities = ["São Paulo", "Rio de Janeiro", "Belo Horizonte", "Curitiba", "Recife"]
    
    return {
        "customer_id": customer_id,
        "name": f"Cliente {customer_id}",
        "email": f"cliente{customer_id}@email.com",
        "city": random.choice(cities),
        "signup_date": (datetime.now() - timedelta(days=random.randint(0, 365))).strftime("%Y-%m-%d"),
        "phone": f"(11) 9{random.randint(1000,9999)}-{random.randint(1000,9999)}",
        # Campo importante para CDC depois:
        "update_ts": datetime.now().isoformat()
    }

In [0]:
# --- 4. EXECUÇÃO DA GERAÇÃO ---

for i in range(NUM_FILES):
    batch_data = [get_random_customer() for _ in range(RECORDS_PER_FILE)]
    
    # Nome do arquivo único baseado no tempo
    file_name = f"customers_{int(time.time())}_{i}.json"
    file_path = f"{raw_path}/{file_name}"
    
    # Escreve o JSON no Volume
    with open(file_path, "w") as f:
        # Simulando formato JSON Lines (um JSON por linha) ou Array of JSON
        # Aqui vamos fazer Array of JSON que é comum em APIs
        json.dump(batch_data, f)
        
    print(f"Arquivo gerado: {file_name}")
    
print("-" * 30)
print(f"✅ Geração concluída! {NUM_FILES} arquivos criados.")